In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
!nvidia-smi

Wed Apr  3 09:35:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   46C    P0             246W / 400W |  12126MiB / 40960MiB |     65%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


### if this is your first time installing the above library, you must restart your kernel and rerun before continuing to the next code

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:

%cd indonlu

/workspace/Tugas Akhir/indonlu


In [7]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer
# %cd utils  
from utils.forward_fn import forward_sequence_classification
from utils.metrics import document_sentiment_metrics_fn
from utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader

In [8]:
from sklearn.metrics import precision_recall_fscore_support

In [9]:
%cd ..

/workspace/Tugas Akhir


### Since this research has an experiment to know the effective representation of text
### and all of representation already precompute, you can choose 1 of below list for an input representation
### `["title_ori", "text_ori", "tc_ori", "filter_word_tc", "trunc_first_short", "trunc_last_short", "trunc_first_long", "trunc_last_long", "trunc_combination",  "textrank", "sum_indobart", "sum_liputan6","sum_pegasus"]`

In [10]:
representasi_teks = 'title_ori'

In [11]:

#####
# Document Sentiment Prosa
#####
from torch.utils.data import Dataset, DataLoader
class DocumentBencanaDataset(Dataset):
    # Static constant variable
    LABEL2INDEX = {'Y': 1, 'N': 0}
    INDEX2LABEL = {0: 'N', 1: 'Y'}
    NUM_LABELS = 2
    
    def load_dataset(self, path): 
        aa  = pd.read_csv(path, sep=',')
        df = aa[[representasi_teks,'label']]
        df.columns = [representasi_teks,'label']
        df['label'] = df['label'].apply(lambda lab: self.LABEL2INDEX[lab])
        return df
    
    def __init__(self, dataset_path, tokenizer, no_special_token=False, *args, **kwargs):
        self.data = self.load_dataset(dataset_path)
        self.tokenizer = tokenizer
        self.no_special_token = no_special_token
    
    def __getitem__(self, index):
        data = self.data.loc[index,:]
        text, sentiment = data[representasi_teks], data['label']
        subwords = self.tokenizer.encode(text, add_special_tokens=not self.no_special_token)
        return np.array(subwords), np.array(sentiment), data[representasi_teks]
    
    def __len__(self):
        return len(self.data)    
        
class DocumentBencanaDataLoader(DataLoader):
    def __init__(self, max_seq_len=256, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.collate_fn = self._collate_fn
        self.max_seq_len = max_seq_len
        
    def _collate_fn(self, batch):
        batch_size = len(batch)
        max_seq_len = max(map(lambda x: len(x[0]), batch))
        max_seq_len = min(self.max_seq_len, max_seq_len)
        
        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)
        
        seq_list = []
        for i, (subwords, sentiment, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            sentiment_batch[i,0] = sentiment
            
            seq_list.append(raw_seq)
            
        return subword_batch, mask_batch, sentiment_batch, seq_list


In [12]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [13]:
# Set random seed
set_seed(28022023)

In [14]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-large-p2')
config = BertConfig.from_pretrained('indobenchmark/indobert-large-p2', hidden_size = 128)
config.num_labels = DocumentBencanaDataset.NUM_LABELS

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-large-p2', config=config,ignore_mismatched_sizes=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 128]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 128]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 128]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

In [15]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, element

In [16]:
%cd dataset

/workspace/Tugas Akhir/dataset


In [17]:
def get_eval(df, bencana):
    act = pd.read_csv(f'./{bencana}/{bencana}_test.csv')
    yact = list(act['label'])
    ypred = list(df['label'])
    p,r, f,s = precision_recall_fscore_support(yact,ypred, average='binary', pos_label='Y')
    print('Evaluasi bencana', bencana)
    print(f"Nilai Precision = {p}")
    print(f"Nilai Recall = {r}")
    print(f"Nilai F1 = {f}")
    return p,r,f

In [40]:
def train_bert(n_epochs, train_loader, valid_loader,optimizer, model, i2w, bencana,test_loader , nomor_run):
    
    for epoch in range(n_epochs):
        model.train()
        torch.set_grad_enabled(True)
    
        total_train_loss = 0
        list_hyp, list_label = [], []

        train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
        for i, batch_data in enumerate(train_pbar):
            # Forward model
            loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

            # Update model
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            tr_loss = loss.item()
            total_train_loss = total_train_loss + tr_loss

            # Calculate metrics
            list_hyp += batch_hyp
            list_label += batch_label

            train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
                total_train_loss/(i+1), get_lr(optimizer)))

        # Calculate train metric
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)
        print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

        # Evaluate on validation
        model.eval()
        torch.set_grad_enabled(False)
        
        total_loss, total_correct, total_labels = 0, 0, 0
        list_hyp, list_label = [], []

        pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
        for i, batch_data in enumerate(pbar):
            batch_seq = batch_data[-1]        
            loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
            
            # Calculate total loss
            valid_loss = loss.item()
            total_loss = total_loss + valid_loss

            # Calculate evaluation metrics
            list_hyp += batch_hyp
            list_label += batch_label
            metrics = document_sentiment_metrics_fn(list_hyp, list_label)

            pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
            
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)
        print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
            total_loss/(i+1), metrics_to_string(metrics)))
    model.eval()
    torch.set_grad_enabled(False)

    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(test_loader, leave=True, total=len(test_loader))
    for i, batch_data in enumerate(pbar):
        _, batch_hyp, _ = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        list_hyp += batch_hyp

    # Save prediction result_classification_Indobert
    df = pd.DataFrame({'label':list_hyp}).reset_index()
    df['id'] = list(pd.read_csv(f'./{bencana}/{bencana}_test.csv')['id'])
    df[['id','label']].to_csv(f'../prediction/result_classification_Indobert/{bencana}_{representasi_teks}_{nomor_run+1}.csv', index=False)
    torch.cuda.empty_cache()
    return get_eval(df, bencana)
    

def run_bert_for_bencana(bencana, nomor_run):
    tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-large-p2')
    config = BertConfig.from_pretrained('indobenchmark/indobert-large-p2', hidden_size = 768)
    config.num_labels = DocumentBencanaDataset.NUM_LABELS

    # Instantiate model
    model_run = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-large-p2', config=config,ignore_mismatched_sizes=True)

    #model_run = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-large-p2', config=config,ignore_mismatched_sizes=True)
    train_dataset_path = f'./{bencana}/{bencana}_train.csv'
    valid_dataset_path = f'./{bencana}/{bencana}_dev.csv'
    test_dataset_path = f'./{bencana}/{bencana}_test.csv'
    train_dataset = DocumentBencanaDataset(train_dataset_path, tokenizer, lowercase=True)
    valid_dataset = DocumentBencanaDataset(valid_dataset_path, tokenizer, lowercase=True)
    test_dataset = DocumentBencanaDataset(test_dataset_path, tokenizer, lowercase=True)

    train_loader = DocumentBencanaDataLoader(dataset=train_dataset, max_seq_len=256, batch_size=24, num_workers=16, shuffle=True)  
    valid_loader = DocumentBencanaDataLoader(dataset=valid_dataset, max_seq_len=256, batch_size=24, num_workers=16, shuffle=False)  
    test_loader = DocumentBencanaDataLoader(dataset=test_dataset, max_seq_len=256, batch_size=24, num_workers=16, shuffle=False)
    w2i, i2w = DocumentBencanaDataset.LABEL2INDEX, DocumentBencanaDataset.INDEX2LABEL
    optimizer = optim.Adam(model_run.parameters(), lr=3e-7)
    model_run = model_run.cuda()
    return train_bert(24, train_loader, valid_loader,optimizer, model_run, i2w, bencana, test_loader, nomor_run )

In [24]:
def display_score_format(fl,pl,rl):
    print(f"Nilai maksimal dari semua run:")
    print("P =", max(pl))
    print("R =", max(rl))
    print("F1 =", max(fl))
    
    print(f"Nilai minimal dari semua run:")
    print("P =", min(pl))
    print("R =", min(rl))
    print("F1 =", min(fl))
    
    print(f"Nilai rata-rata dari semua run:")
    print("P =", sum(pl)/len(pl))
    print("R =", sum(rl)/len(rl))
    print("F1 =", sum(fl)/len(fl))
    
    

In [39]:
f1_list, p_list, r_list = [],[],[]
for i in range(3):
    print(f"run ke {i+1}")
    p,r,f = run_bert_for_bencana('angin_topan',i)
    f1_list.append(f)
    p_list.append(p)
    r_list.append(r)
display_score_format(f1_list,p_list,r_list)

run ke 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 768]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 768]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

(Epoch 1) TRAIN LOSS:0.4784 LR:0.00000100: 100%|██████████| 38/38 [00:03<00:00,  9.92it/s]


(Epoch 1) TRAIN LOSS:0.4784 ACC:0.82 F1:0.01 REC:0.01 PRE:0.14 LR:0.00000100


VALID LOSS:0.6719 ACC:0.68 F1:0.00 REC:0.00 PRE:0.00: 100%|██████████| 5/5 [00:01<00:00,  3.79it/s]


...

(Epoch 24) TRAIN LOSS:0.1211 ACC:0.95 F1:0.87 REC:0.86 PRE:0.88 LR:0.00000100


VALID LOSS:0.3034 ACC:0.90 F1:0.83 REC:0.75 PRE:0.92: 100%|██████████| 5/5 [00:01<00:00,  3.26it/s]


(Epoch 24) VALID LOSS:0.3034 ACC:0.90 F1:0.83 REC:0.75 PRE:0.92


100%|██████████| 9/9 [00:01<00:00,  6.24it/s]

Evaluasi bencana angin_topan
Nilai Precision = 0.9
Nilai Recall = 0.782608695652174
Nilai F1 = 0.8372093023255814
Nilai maksimal dari semua run:
P = 0.9
R = 0.782608695652174
F1 = 0.8372093023255814
Nilai minimal dari semua run:
P = 0.84
R = 0.45652173913043476
F1 = 0.5915492957746479
Nilai rata-rata dari semua run:
P = 0.8729292929292929
R = 0.6231884057971014
F1 = 0.7209786044300343


In [22]:
f1_list, p_list, r_list = [],[],[]
for i in range(3):
    print(f"run ke {i+1}")
    p,r,f = run_bert_for_bencana('banjir',i)
    f1_list.append(f)
    p_list.append(p)
    r_list.append(r)
display_score_format(f1_list,p_list,r_list)

run ke 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 128]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 128]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 128]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

(Epoch 1) TRAIN LOSS:0.5599 LR:0.00000300: 100%|██████████| 39/39 [00:08<00:00,  4.59it/s]


(Epoch 1) TRAIN LOSS:0.5599 ACC:0.76 F1:0.00 REC:0.00 PRE:0.00 LR:0.00000300


VALID LOSS:0.7569 ACC:0.58 F1:0.00 REC:0.00 PRE:0.00: 100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


...

(Epoch 24) TRAIN LOSS:0.3604 ACC:0.87 F1:0.69 REC:0.61 PRE:0.80 LR:0.00000300


VALID LOSS:0.5677 ACC:0.75 F1:0.69 REC:0.67 PRE:0.71: 100%|██████████| 5/5 [00:14<00:00,  2.86s/it]


(Epoch 24) VALID LOSS:0.5677 ACC:0.75 F1:0.69 REC:0.67 PRE:0.71


100%|██████████| 9/9 [00:12<00:00,  1.42s/it]


Evaluasi bencana banjir
Nilai Precision = 0.6226415094339622
Nilai Recall = 0.6111111111111112
Nilai F1 = 0.616822429906542
Nilai maksimal dari semua run:
P = 0.6818181818181818
R = 0.6111111111111112
F1 = 0.616822429906542
Nilai minimal dari semua run:
P = 0.5833333333333334
R = 0.2777777777777778
F1 = 0.39473684210526316
Nilai rata-rata dari semua run:
P = 0.6292643415284925
R = 0.425925925925926
F1 = 0.4927419795594907


In [21]:
f1_list, p_list, r_list = [],[],[]
for i in range(3):
    print(f"run ke {i+1}")
    p,r,f = run_bert_for_bencana('erupsi',i)
    f1_list.append(f)
    p_list.append(p)
    r_list.append(r)
display_score_format(f1_list,p_list,r_list)

run ke 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 128]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 128]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 128]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

(Epoch 1) TRAIN LOSS:0.4671 LR:0.00000300: 100%|██████████| 33/33 [00:31<00:00,  1.04it/s]


(Epoch 1) TRAIN LOSS:0.4671 ACC:0.87 F1:0.00 REC:0.00 PRE:0.00 LR:0.00000300


VALID LOSS:0.4908 ACC:0.82 F1:0.00 REC:0.00 PRE:0.00: 100%|██████████| 5/5 [00:05<00:00,  1.03s/it]


...

(Epoch 24) TRAIN LOSS:0.3107 ACC:0.89 F1:0.41 REC:0.33 PRE:0.56 LR:0.00000300


VALID LOSS:0.3973 ACC:0.82 F1:0.18 REC:0.11 PRE:0.50: 100%|██████████| 5/5 [00:16<00:00,  3.30s/it]


(Epoch 24) VALID LOSS:0.3973 ACC:0.82 F1:0.18 REC:0.11 PRE:0.50


100%|██████████| 9/9 [00:16<00:00,  1.89s/it]


Evaluasi bencana erupsi
Nilai Precision = 0.8
Nilai Recall = 0.11764705882352941
Nilai F1 = 0.20512820512820512
Nilai maksimal dari semua run:
P = 0.875
R = 0.20588235294117646
F1 = 0.33333333333333337
Nilai minimal dari semua run:
P = 0.0
R = 0.0
F1 = 0.0
Nilai rata-rata dari semua run:
P = 0.5583333333333333
R = 0.10784313725490195
F1 = 0.17948717948717952


In [22]:
f1_list, p_list, r_list = [],[],[]
for i in range(3):
    print(f"run ke {i+1}")
    p,r,f = run_bert_for_bencana('gempa',i)
    f1_list.append(f)
    p_list.append(p)
    r_list.append(r)
display_score_format(f1_list,p_list,r_list)

run ke 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 128]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 128]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 128]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

(Epoch 1) TRAIN LOSS:0.5131 LR:0.00000300: 100%|██████████| 39/39 [00:46<00:00,  1.20s/it]


(Epoch 1) TRAIN LOSS:0.5131 ACC:0.81 F1:0.01 REC:0.01 PRE:1.00 LR:0.00000300


VALID LOSS:0.6208 ACC:0.65 F1:0.00 REC:0.00 PRE:0.00: 100%|██████████| 5/5 [00:19<00:00,  3.88s/it]


...

(Epoch 24) TRAIN LOSS:0.3661 ACC:0.87 F1:0.58 REC:0.47 PRE:0.75 LR:0.00000300


VALID LOSS:0.5246 ACC:0.73 F1:0.43 REC:0.29 PRE:0.83: 100%|██████████| 5/5 [00:17<00:00,  3.58s/it]


(Epoch 24) VALID LOSS:0.5246 ACC:0.73 F1:0.43 REC:0.29 PRE:0.83


100%|██████████| 9/9 [00:18<00:00,  2.08s/it]


Evaluasi bencana gempa
Nilai Precision = 0.9090909090909091
Nilai Recall = 0.2222222222222222
Nilai F1 = 0.3571428571428571
Nilai maksimal dari semua run:
P = 0.9090909090909091
R = 0.4666666666666667
F1 = 0.5526315789473684
Nilai minimal dari semua run:
P = 0.6774193548387096
R = 0.2222222222222222
F1 = 0.3389830508474576
Nilai rata-rata dari semua run:
P = 0.7669319927384444
R = 0.3037037037037037
F1 = 0.41625249564589434


In [ ]:
f1_list, p_list, r_list = [],[],[]
for i in range(3):
    print(f"run ke {i+1}")
    p,r,f = run_bert_for_bencana('karhutla',i)
    f1_list.append(f)
    p_list.append(p)
    r_list.append(r)
display_score_format(f1_list,p_list,r_list)

run ke 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 128]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 128]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 128]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

(Epoch 1) TRAIN LOSS:0.6660 LR:0.00000300: 100%|██████████| 36/36 [00:52<00:00,  1.47s/it]


(Epoch 1) TRAIN LOSS:0.6660 ACC:0.63 F1:0.04 REC:0.02 PRE:0.19 LR:0.00000300


VALID LOSS:0.7050 ACC:0.55 F1:0.00 REC:0.00 PRE:0.00: 100%|██████████| 5/5 [00:19<00:00,  3.88s/it]


...

VALID LOSS:0.6296 ACC:0.68 F1:0.54 REC:0.42 PRE:0.77: 100%|██████████| 5/5 [00:16<00:00,  3.38s/it]


(Epoch 21) VALID LOSS:0.6296 ACC:0.68 F1:0.54 REC:0.42 PRE:0.77


(Epoch 22) TRAIN LOSS:0.5249 LR:0.00000300: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it]


(Epoch 22) TRAIN LOSS:0.5249 ACC:0.77 F1:0.62 REC:0.57 PRE:0.70 LR:0.00000300


  0%|          | 0/5 [00:00<?, ?it/s]

In [21]:
f1_list, p_list, r_list = [],[],[]
for i in range(1,3):
    print(f"run ke {i+1}")
    p,r,f = run_bert_for_bencana('kekeringan',i)
    f1_list.append(f)
    p_list.append(p)
    r_list.append(r)
display_score_format(f1_list,p_list,r_list)

run ke 2


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 128]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 128]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 128]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

(Epoch 1) TRAIN LOSS:0.5445 LR:0.00000300: 100%|██████████| 34/34 [00:39<00:00,  1.16s/it]


(Epoch 1) TRAIN LOSS:0.5445 ACC:0.79 F1:0.01 REC:0.01 PRE:0.25 LR:0.00000300


VALID LOSS:0.5940 ACC:0.72 F1:0.00 REC:0.00 PRE:0.00: 100%|██████████| 5/5 [00:10<00:00,  2.02s/it]


...

(Epoch 24) TRAIN LOSS:0.3210 ACC:0.91 F1:0.76 REC:0.72 PRE:0.80 LR:0.00000300


VALID LOSS:0.4650 ACC:0.83 F1:0.64 REC:0.54 PRE:0.79: 100%|██████████| 5/5 [00:10<00:00,  2.08s/it]


(Epoch 24) VALID LOSS:0.4650 ACC:0.83 F1:0.64 REC:0.54 PRE:0.79


100%|██████████| 9/9 [00:12<00:00,  1.38s/it]


Evaluasi bencana kekeringan
Nilai Precision = 0.8571428571428571
Nilai Recall = 0.3076923076923077
Nilai F1 = 0.45283018867924535
Nilai maksimal dari semua run:
P = 0.8571428571428571
R = 0.6666666666666666
F1 = 0.619047619047619
Nilai minimal dari semua run:
P = 0.5777777777777777
R = 0.3076923076923077
F1 = 0.45283018867924535
Nilai rata-rata dari semua run:
P = 0.7174603174603174
R = 0.48717948717948717
F1 = 0.5359389038634321


In [22]:
f1_list, p_list, r_list = [],[],[]
for i in range(3):
    print(f"run ke {i+1}")
    p,r,f = run_bert_for_bencana('longsor',i)
    f1_list.append(f)
    p_list.append(p)
    r_list.append(r)
display_score_format(f1_list,p_list,r_list)

run ke 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 128]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 128]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 128]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

(Epoch 1) TRAIN LOSS:0.5978 LR:0.00000300: 100%|██████████| 39/39 [00:43<00:00,  1.11s/it]


(Epoch 1) TRAIN LOSS:0.5978 ACC:0.73 F1:0.11 REC:0.07 PRE:0.25 LR:0.00000300


VALID LOSS:0.7413 ACC:0.59 F1:0.00 REC:0.00 PRE:0.00: 100%|██████████| 5/5 [00:14<00:00,  2.96s/it]


...

(Epoch 24) TRAIN LOSS:0.3596 ACC:0.87 F1:0.70 REC:0.64 PRE:0.79 LR:0.00000300


VALID LOSS:0.4948 ACC:0.78 F1:0.65 REC:0.49 PRE:0.95: 100%|██████████| 5/5 [00:13<00:00,  2.72s/it]


(Epoch 24) VALID LOSS:0.4948 ACC:0.78 F1:0.65 REC:0.49 PRE:0.95


100%|██████████| 9/9 [00:14<00:00,  1.58s/it]


Evaluasi bencana longsor
Nilai Precision = 0.7105263157894737
Nilai Recall = 0.5094339622641509
Nilai F1 = 0.5934065934065935
Nilai maksimal dari semua run:
P = 0.7307692307692307
R = 0.5094339622641509
F1 = 0.5934065934065935
Nilai minimal dari semua run:
P = 0.7096774193548387
R = 0.3584905660377358
F1 = 0.4810126582278481
Nilai rata-rata dari semua run:
P = 0.7169909886378477
R = 0.42767295597484284
F1 = 0.5327429251479884


In [23]:
f1_list, p_list, r_list = [],[],[]
for i in range(3):
    print(f"run ke {i+1}")
    p,r,f = run_bert_for_bencana('tsunami',i)
    f1_list.append(f)
    p_list.append(p)
    r_list.append(r)
display_score_format(f1_list,p_list,r_list)

run ke 1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized because the shapes did not match:
- embeddings.word_embeddings.weight: found shape torch.Size([30522, 1024]) in the checkpoint and torch.Size([30522, 128]) in the model instantiated
- embeddings.position_embeddings.weight: found shape torch.Size([512, 1024]) in the checkpoint and torch.Size([512, 128]) in the model instantiated
- embeddings.token_type_embeddings.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([2, 128]) in the model instantiated
- embeddings.LayerNorm.weight: found shape torch.Size([1024]) i

(Epoch 1) TRAIN LOSS:0.5904 LR:0.00000300: 100%|██████████| 37/37 [00:32<00:00,  1.13it/s]


(Epoch 1) TRAIN LOSS:0.5904 ACC:0.73 F1:0.01 REC:0.00 PRE:0.33 LR:0.00000300


VALID LOSS:0.6615 ACC:0.63 F1:0.00 REC:0.00 PRE:0.00: 100%|██████████| 5/5 [00:12<00:00,  2.54s/it]


...

(Epoch 24) TRAIN LOSS:0.4168 ACC:0.84 F1:0.68 REC:0.62 PRE:0.75 LR:0.00000300


VALID LOSS:0.5630 ACC:0.75 F1:0.68 REC:0.70 PRE:0.65: 100%|██████████| 5/5 [00:15<00:00,  3.16s/it]


(Epoch 24) VALID LOSS:0.5630 ACC:0.75 F1:0.68 REC:0.70 PRE:0.65


100%|██████████| 9/9 [00:15<00:00,  1.73s/it]


Evaluasi bencana tsunami
Nilai Precision = 0.5070422535211268
Nilai Recall = 0.75
Nilai F1 = 0.6050420168067226
Nilai maksimal dari semua run:
P = 0.5833333333333334
R = 0.8541666666666666
F1 = 0.6050420168067226
Nilai minimal dari semua run:
P = 0.44086021505376344
R = 0.4375
F1 = 0.5
Nilai rata-rata dari semua run:
P = 0.5104119339694079
R = 0.6805555555555555
F1 = 0.5622007668315554
